This code is to rasterize OSM road networks into images.

In [1]:
'''
Author: an.tran@grab.com
'''
import rasterio
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import osr
import cv2
from skeleton import build_graph
from shapely import wkt
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from shapely.ops import transform
from shapely.geometry import box
import geojson
from skimage.morphology import binary_dilation, binary_erosion
from skimage.morphology import skeletonize
import sknw
import math
import geohash
import osmnx as ox
import affine
import cv2
import tifffile as tiff
from joblib import Parallel, delayed

In [2]:
def get_lon_lat_from_image_name(img_name, transformation_file, r=0, c=0):
    '''Transform image name to lat,lon of upper-left corner. The computation backward does not work well.
    
    Parameters:
    -----------
    img_name: string
        Image file name
    transformation_file: string
        Pickle file containing transformation matrix
    r, c: int
        Row and column image coordinates of the point we want to get lon, lat coordinates

    Returns:
    --------
    ls: list
        A new list of linestring in lon, lat.
    '''
    trunk = img_name.split('_')
    city_id = '_'.join(trunk[0:-4])
    sat_id = '%s.tif' % ('_'.join(trunk[0:-3]))
    r += int(trunk[-3])
    c += int(trunk[-2])
    df = pd.read_pickle(transformation_file)
    row = df.loc[df.loc[:, 'TifImageName'] == sat_id, :]
    transform_matrix = row.values[0, 1]
    #print('transform matrix: ', transform_matrix)
    crs_number = row.values[0, 2]
    src = osr.SpatialReference()
    src.ImportFromEPSG(crs_number)
    tgt = osr.SpatialReference()
    tgt.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(src, tgt)
    x, y = transform_matrix * (c+0.5, r+0.5)
    lon, lat = transform.TransformPoint(x, y)[:2]
    return lon, lat

In [3]:
def get_image_coordinate_from_lon_lat(img_name, transformation_file, lon, lat):
    '''The function to convert lon, lat coordinates to image coordinates.
    
    Parameters:
    -----------
    img_name: string
        Image file name
    transformation_file: string
        Pickle file containing transformation matrix
    lon, lat: float
        Longitude and latitude

    Returns:
    --------
    r, c: int
        Row and column image coordinates of the point we want to get lon, lat coordinates
    '''
    trunk = img_name.split('_')
    city_id = '_'.join(trunk[0:-4])
    sat_id = '%s.tif' % ('_'.join(trunk[0:-3]))
    r0 = int(trunk[-3])
    c0 = int(trunk[-2])
    df = pd.read_pickle(transformation_file)
    row = df.loc[df.loc[:, 'TifImageName'] == sat_id, :]
    transform_matrix = row.values[0, 1]
    #print('transform matrix: ', transform_matrix)
    crs_number = row.values[0, 2]
    tgt = osr.SpatialReference()
    tgt.ImportFromEPSG(crs_number)
    src = osr.SpatialReference()
    src.ImportFromEPSG(4326)
    transform = osr.CoordinateTransformation(src, tgt)
    x, y = transform.TransformPoint(lon, lat)[:2]
    A = transform_matrix
    ta = np.array([[A.a, A.b, A.c], [A.d, A.e, A.f], [0, 0, 1]])
    Ainv = np.linalg.inv(np.array(ta))
    Ain = affine.Affine(Ainv[0][0], Ainv[0][1], Ainv[0][2], Ainv[1][0], Ainv[1][1], Ainv[1][2])
    c, r = Ain * (x, y)
    r = r - r0 - 0.5
    c = c - c0 - 0.5
    return c, r

In [4]:
sat_id = 'VM_Tan_An_18Q3_V0' # satellite id
root_folder = '/Users/an.tran/Desktop/map-workspace/data/digital-globe'
jpg_image_folder = os.path.join(root_folder, sat_id, 'jpg')
osm_out_folder = os.path.join(root_folder, sat_id, 'osm_opencv')
if not os.path.exists(osm_out_folder):
    os.makedirs(osm_out_folder)
transformation_file = os.path.join(root_folder, city_id, '%s.pkl'%city_id)

In [5]:
jpg_images = glob.glob(os.path.join(jpg_image_folder, '*.jpg'))
jpg_images.sort()
len(jpg_images)

26613

In [6]:
def rasterize_osm_road_network(img_name, G, transformation_file, num_lanes_profile, lane_width=8):
    '''Rastering road network in graph G to image using OpenCV.
    
    Parameters:
    -----------
    img_name: string
        Jpg input image
    G : ox.DiGraph
        OSM road network
    img_size: tuple
        Image size, default: (1024, 1024)
    transformation_file: string
        Pickle file containing transformation matrix
    num_lanes_profile: dict
        Dictionary of number of lanes per highway road type.
    lane_width: int
        Lane width
    
    Returns:
    --------
    None
    '''
    image = np.zeros((1024, 1024))
    for (s,e,data) in G.edges(data=True):
        lat_s = G.node[s]['y']
        lon_s = G.node[s]['x']
        c_s, r_s = get_image_coordinate_from_lon_lat(img_name, transformation_file, lon=lon_s, lat=lat_s)
        lat_e = G.node[e]['y']
        lon_e = G.node[e]['x']
        c_e, r_e = get_image_coordinate_from_lon_lat(img_name, transformation_file, lon=lon_e, lat=lat_e)
        c_s = int(np.around(c_s))
        r_s = int(np.around(r_s))
        c_e = int(np.around(c_e))
        r_e = int(np.around(r_e))
        if data['highway'] in num_lanes_profile:
            num_lanes = num_lanes_profile[data['highway']]
        else: num_lanes = 1
        if 'lanes' in data.keys():
            try:
                num_lanes = int(data['lanes'])
            except ValueError:
                pass

        cv2.line(image, (c_s, r_s), (c_e, r_e), color=(255), thickness=int(lane_width*num_lanes))
    return image

In [7]:
# encoding different num_lanes values
num_lanes_profile = {'motorway':1.5, 'motorway_link':1.5, 'trunk':1.5, 'trunk_link':1.5, 
                     'primary':1.5, 'primary_link':1.5, 'secondary':1.5, 'secondary_link':1.5,
                     'tertiary':1.5, 'tertiary_link':1.5, 'residential':1, 'service':1,
                     'unclassified':1, 'track':0.5, 'living_street':0.5}
                     #'footway':0.2, 'path':0.2, 'steps':0.2, 'pedestrian':0.2

In [8]:
# using try-except to handle exceptions
def rasterize_osm_using_opencv(input_path, osm_out_folder, transformation_file, epsilon=1e-3):
    """Rasterize OSM road network using opencv, and write it to the result folder.

    Parameters
    ----------
    input_image: string
        Jpg input image
    osm_out_folder : string
        OSM output folder
    transformation_file: string
        Pickle file containing transformation matrix

    Returns
    -------
    None
    """
    img_name = os.path.basename(input_path)
    out_file_name = '%s_mask.png' % (img_name[:-8])
    out_file = os.path.join(osm_out_folder, out_file_name)
    if os.path.exists(out_file):
        return
    left, top = get_lon_lat_from_image_name(img_name, transformation_file)
    right, bottom = get_lon_lat_from_image_name(img_name, transformation_file, r=1024, c=1024)
    try:
        G = ox.graph_from_bbox(north=top+epsilon, south=bottom-epsilon, west=left-epsilon, east=right+epsilon, network_type='all_private',
                    simplify=False, retain_all=True, truncate_by_edge=False)
    except ox.EmptyOverpassResponse:
        cv2.imwrite(out_file, np.zeros((1024, 1024)))
        return
    except: # other exception
        return
    mask = rasterize_osm_road_network(img_name, G, num_lanes_profile, lane_width=8)
    cv2.imwrite(out_file, mask)
    return

In [9]:
# for input_image in jpg_images:
#     rasterize_osm_using_opencv(input_image, osm_out_folder)
Parallel(n_jobs=-3, backend='threading')(delayed(rasterize_osm_using_opencv)(input_image, osm_out_folder, transformation_file) for input_image in jpg_images)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,